In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Load your API key from .env file
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Test to confirm
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API key loaded successfully!")
else:
    print("❌ API key not found. Please check your .env file.")

✅ OpenAI API key loaded successfully!


In [3]:
# Load nutrition dataset
data_path = r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv"

try:
    df = pd.read_csv(data_path)
    print("✅ Nutrition dataset loaded successfully!")
    display(df.head())
except FileNotFoundError:
    print("❌ Nutrition data file not found. Please check the path:", data_path)

✅ Nutrition dataset loaded successfully!


,Food Item,Category,Subcategory,Health Suitability,Calories,Carbs,Protein,Fats,Fiber,Vitamin A,Vitamin C,Iron,Calcium,Local Availability
0,Oats,Breakfast,Grains,"Diabetes, Obesity, PCOD, Thyroid",389,66.0,16.0,7.0,11.0,0,0.0,4.0,54,Yes
1,Lentils,Lunch,Protein,"Diabetes, Anemia, Thyroid",116,20.0,9.0,0.4,8.0,8,1.0,3.3,19,Yes
2,Banana,Breakfast,Fruit,"Diabetes (in moderation), Anemia",89,23.0,1.0,0.3,2.6,64,8.7,0.3,5,Yes
3,Paneer,Lunch,Dairy,"Kidney issues (moderate), PCOD, Obesity",265,1.2,18.0,20.0,0.0,9,0.0,0.7,208,Yes
4,Brown Rice,Lunch,Grains,"Diabetes, Thyroid, Heart issues",123,26.0,2.6,1.0,1.8,0,0.0,0.4,3,Yes


In [4]:
# Prompt to test Generative AI response
prompt = """
You are a professional diet planner. Generate a healthy 1-day Indian meal plan 
for a diabetic person in South India. Use only common local foods like dosa, idli, 
millets, pulses, vegetables, and fruits. Include breakfast, lunch, and dinner 
with short explanations and total nutritional values (Calories, Carbs, Proteins, Fat).
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7
)

print(response.choices[0].message.content)

Here's a healthy 1-day Indian meal plan for a diabetic person in South India, focusing on common local foods like dosa, idli, millets, pulses, vegetables, and fruits. The meal plan is designed to provide balanced nutrition while managing blood sugar levels.

### 1-Day Meal Plan

#### Breakfast
- **Ragi Idli with Sambar**
  - **Ingredients**: Ragi flour (fermented), urad dal (black gram), and mixed vegetables (carrots, beans, and peas) in sambar.
  - **Description**: Ragi is a high-fiber, low-GI grain that helps in managing blood sugar levels. Sambar, made with lentils and vegetables, adds protein and fiber, promoting satiety.
  
#### Nutritional Values for Breakfast:
- Calories: 250
- Carbs: 40g
- Protein: 10g
- Fat: 5g

---

#### Mid-Morning Snack
- **Cucumber and Tomato Salad**
  - **Ingredients**: Chopped cucumber, tomatoes, onions, and a sprinkle of lemon juice.
  - **Description**: This salad is refreshing, low in calories, and rich in vitamins and minerals.

#### Nutritional Valu

In [8]:
import pandas as pd
import random
from openai import OpenAI
from IPython.display import display

# Initialize OpenAI client
# client = OpenAI(api_key="OPENAI_API_KEY")  # Replace with your API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Load dataset
df = pd.read_csv(r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv")

meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# User input
age = int(input("Enter your age: "))
gender = input("Enter your gender (M/F/Other): ")
health_condition = input("Enter your health condition (e.g., Diabetes, BP, PCOD, Thyroid, Obesity): ")

# --- Helper functions ---
def filter_foods(category, health_condition):
    """Filter foods by category and health suitability"""
    foods = df[
        (df['Category'] == category) &
        (df['Health Suitability'].str.contains(health_condition, case=False, na=False))
    ]
    if len(foods) == 0:
        foods = df[df['Category'] == category]
    return foods

def generate_explanation(food_item, category, health_condition):
    """Use OpenAI API to generate explanation for each meal"""
    prompt = f"""
    You are a professional Indian dietitian.
    Explain in 2-3 sentences why including {food_item} for {category} 
    is beneficial for someone with {health_condition}.
    Keep it simple and friendly.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or "gpt-4o" if you have access
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    
    return response.choices[0].message.content.strip()

def generate_meal(category, health_condition):
    """Generate one meal and AI explanation"""
    foods = filter_foods(category, health_condition)
    meal_item = foods.sample(n=1).iloc[0]
    
    explanation = generate_explanation(meal_item['Food Item'], category, health_condition)
    
    return {
        'Food Item': meal_item['Food Item'],
        'Carbs': meal_item['Carbs'],
        'Protein': meal_item['Protein'],
        'Fat': meal_item['Fats'],
        'Calories': meal_item['Calories'],
        'Explanation': explanation
    }

# --- Generate 7-day plan ---
weekly_plan = []
for day in range(1, 8):
    daily_plan = {'Day': f'Day {day}'}
    total_calories = 0
    day_explanations = []

    for meal in meal_types:
        meal_info = generate_meal(meal, health_condition)
        daily_plan[meal] = f"{meal_info['Food Item']} (Carbs: {meal_info['Carbs']}g, Protein: {meal_info['Protein']}g, Fat: {meal_info['Fat']}g, {meal_info['Calories']} kcal)"
        total_calories += meal_info['Calories']
        day_explanations.append(meal_info['Explanation'])
    
    daily_plan['Total Calories'] = total_calories
    daily_plan['Explanation'] = " ".join(day_explanations)
    weekly_plan.append(daily_plan)

# --- Display & Save ---
weekly_plan_df = pd.DataFrame(weekly_plan)
pd.set_option('display.max_columns', None)
display(
    weekly_plan_df.style.set_properties(**{'text-align': 'left'}).set_table_styles(
        [{'selector': 'th', 'props': [('text-align', 'left')]}]
    )
)

weekly_plan_df.to_csv('7_day_meal_plan.csv', index=False)
print("\n✅ 7-day meal plan saved as '7_day_meal_plan.csv'")

Enter your age:  23
Enter your gender (M/F/Other):  F
Enter your health condition (e.g., Diabetes, BP, PCOD, Thyroid, Obesity):  PCOD


,Day,Breakfast,Lunch,Dinner,Snack,Total Calories,Explanation
0,Day 1,"Yogurt (Carbs: 3.6g, Protein: 10.0g, Fat: 0.4g, 59 kcal)","Chickpeas (Carbs: 27.0g, Protein: 9.0g, Fat: 2.6g, 164 kcal)","Pumpkin (Carbs: 6.5g, Protein: 1.0g, Fat: 0.1g, 26 kcal)","Flax Seeds (Carbs: 29.0g, Protein: 18.0g, Fat: 42.0g, 534 kcal)",783,"Including yogurt for breakfast is beneficial for someone with PCOD because it is rich in probiotics, which can help balance gut health and support digestion. Additionally, yogurt is a good source of protein and calcium, helping to keep you full longer and stabilize blood sugar levels, which is important for managing PCOD symptoms. Plus, it's versatile and can be easily paired with fruits or nuts for a nutritious start to your day! Including chickpeas in your lunch is great for managing PCOD because they are high in fiber, which helps regulate blood sugar levels and keeps you feeling full longer. They also provide essential nutrients and protein, supporting healthy hormone balance and aiding in weight management. Plus, they can be easily added to salads or curries for a delicious and nutritious meal! Including pumpkin in your dinner can be beneficial for someone with PCOD because it's low in calories and high in fiber, which helps manage weight and keeps you feeling full longer. Additionally, pumpkin is rich in antioxidants and vitamins, particularly vitamin A, which can help regulate hormone levels and support overall reproductive health. Enjoying pumpkin can be a tasty and nutritious way to support your health! Flax seeds are a great snack choice for someone with PCOD because they are rich in omega-3 fatty acids, which can help reduce inflammation and balance hormones. They also contain high levels of fiber, which aids digestion and helps manage blood sugar levels, making them a smart addition to your diet. Enjoying flax seeds can support overall health and help you feel your best!"
1,Day 2,"Millets (Carbs: 73.0g, Protein: 11.0g, Fat: 4.0g, 378 kcal)","Methi (Fenugreek) (Carbs: 7.0g, Protein: 4.0g, Fat: 0.8g, 49 kcal)","Spinach (Carbs: 3.6g, Protein: 2.9g, Fat: 0.4g, 23 kcal)","Almonds (Carbs: 22.0g, Protein: 21.0g, Fat: 50.0g, 579 kcal)",1029,"Including millets in breakfast is great for managing PCOD because they are high in fiber, which helps regulate blood sugar levels and reduces insulin resistance. Additionally, millets are rich in essential nutrients and can help keep you full longer, aiding in weight management. So, starting your day with millets can be a tasty and healthful choice! Including Methi (Fenugreek) in your lunch is great for managing PCOD because it helps regulate blood sugar levels and improve insulin sensitivity, which can support weight management. Additionally, its high fiber content aids digestion and may help reduce inflammation, making it a wonderful addition to your meals! Including spinach in dinner can be really beneficial for someone with PCOD because it's packed with essential nutrients like iron, magnesium, and vitamins that help regulate hormones and improve insulin sensitivity. Plus, its high fiber content aids digestion and keeps you feeling full, which can help manage weight effectively. So, it's a delicious way to support your health! Including almonds as a snack is great for someone with PCOD because they are rich in healthy fats, fiber, and protein, which help keep you full and stabilize blood sugar levels. Additionally, almonds contain magnesium, which can help manage insulin resistance and support hormonal balance. Enjoying a handful of almonds can be a delicious and nutritious choice!"
2,Day 3,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g, 155 kcal)","Paneer (Carbs: 1.2g, Protein: 18.0g, Fat: 20.0g, 265 kcal)","Chicken Breast (Carbs: 0.0g, Protein: 31.0g, Fat: 3.6g, 165 kcal)","Flax Seeds (Carbs: 29.0g, Protein: 18.0g, Fat: 42.0g, 534 kcal)",1119,"Including eggs for breakfast is beneficial for someone with PCOD because they are a great source of high-quality protein an


✅ 7-day meal plan saved as '7_day_meal_plan.csv'


In [9]:
import os
import pandas as pd
import random
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display

# --- Initialize OpenAI Client ---
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- Load Dataset ---
df = pd.read_csv(r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv")

meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# --- User Input ---
age = int(input("Enter your age: "))
gender = input("Enter your gender (M/F/Other): ")
health_condition = input("Enter your health condition (e.g., Diabetes, BP, PCOD, Thyroid, Obesity): ")

# --- Helper Functions ---
def filter_foods(category, health_condition):
    """Filter foods by category and health suitability"""
    foods = df[
        (df['Category'] == category) &
        (df['Health Suitability'].str.contains(health_condition, case=False, na=False))
    ]
    if len(foods) == 0:
        foods = df[df['Category'] == category]
    return foods

def generate_explanation(food_item, category, health_condition):
    """Use OpenAI API to generate explanation for each meal"""
    prompt = f"""
    You are a professional Indian dietitian.
    Explain in 2-3 sentences why including {food_item} for {category} 
    is beneficial for someone with {health_condition}.
    Keep it simple and friendly.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # use gpt-4o-mini for faster response
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    
    return response.choices[0].message.content.strip()

def generate_meal(category, health_condition):
    """Generate one meal and AI explanation"""
    foods = filter_foods(category, health_condition)
    meal_item = foods.sample(n=1).iloc[0]
    
    explanation = generate_explanation(meal_item['Food Item'], category, health_condition)
    
    return {
        'Food Item': meal_item['Food Item'],
        'Carbs': meal_item['Carbs'],
        'Protein': meal_item['Protein'],
        'Fat': meal_item['Fats'],
        'Calories': meal_item['Calories'],
        'Explanation': explanation
    }

# --- Generate 1-Day Plan ---
daily_plan = {'Day': 'Day 1'}
total_calories = 0
day_explanations = []

for meal in meal_types:
    meal_info = generate_meal(meal, health_condition)
    daily_plan[meal] = f"{meal_info['Food Item']} (Carbs: {meal_info['Carbs']}g, Protein: {meal_info['Protein']}g, Fat: {meal_info['Fat']}g, {meal_info['Calories']} kcal)"
    total_calories += meal_info['Calories']
    day_explanations.append(meal_info['Explanation'])

daily_plan['Total Calories'] = total_calories
daily_plan['Explanation'] = " ".join(day_explanations)

# --- Display & Save ---
one_day_plan_df = pd.DataFrame([daily_plan])
pd.set_option('display.max_columns', None)

display(
    one_day_plan_df.style.set_properties(**{'text-align': 'left'}).set_table_styles(
        [{'selector': 'th', 'props': [('text-align', 'left')]}]
    )
)

one_day_plan_df.to_csv('1_day_meal_plan.csv', index=False)
print("\n✅ 1-day meal plan saved as '1_day_meal_plan.csv'")

Enter your age:  23
Enter your gender (M/F/Other):  F
Enter your health condition (e.g., Diabetes, BP, PCOD, Thyroid, Obesity):  PCOD


,Day,Breakfast,Lunch,Dinner,Snack,Total Calories,Explanation
0,Day 1,"Millets (Carbs: 73.0g, Protein: 11.0g, Fat: 4.0g, 378 kcal)","Quinoa (Carbs: 64.0g, Protein: 14.0g, Fat: 6.0g, 368 kcal)","Mushroom (Carbs: 3.3g, Protein: 3.1g, Fat: 0.3g, 22 kcal)","Almonds (Carbs: 22.0g, Protein: 21.0g, Fat: 50.0g, 579 kcal)",1347,"Including millets for breakfast is great for managing PCOD because they are rich in fiber, which helps regulate blood sugar levels and keeps you feeling full longer. Millets also contain essential nutrients and antioxidants that can support hormonal balance, making them a healthy and nourishing choice to kickstart your day! Including quinoa in lunch is beneficial for someone with PCOD because it's a great source of protein and fiber, which helps keep blood sugar levels stable and promotes feelings of fullness. Additionally, quinoa is rich in essential nutrients and antioxidants that can support overall hormonal balance, making it a smart choice for managing PCOD symptoms. Plus, it's versatile and easy to incorporate into various dishes! Including mushrooms for dinner can be beneficial for someone with PCOD because they are low in calories and high in fiber, which helps with weight management and keeps you feeling full. Additionally, mushrooms are rich in antioxidants and vitamins that can support hormonal balance and overall health. They also have anti-inflammatory properties, which can be helpful in managing PCOD symptoms. Including almonds as a snack is beneficial for someone with PCOD because they are rich in healthy fats, protein, and fiber, which can help stabilize blood sugar levels and reduce cravings. Additionally, almonds are packed with essential nutrients like magnesium and vitamin E, which can support hormonal balance and overall reproductive health. Enjoying a handful of almonds can be a tasty and nutritious way to support your wellness journey!"



✅ 1-day meal plan saved as '1_day_meal_plan.csv'


In [15]:
import os
import json
import pandas as pd
import random
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display

# Load OpenAI API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Dataset path
dataset_path = r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv"
previous_plans_path = r"C:\Users\polam\Downloads\AI Diet Planner\data\previous_1day_plans.json"

# Load dataset
df = pd.read_csv(dataset_path)

meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# User input
age = int(input("Enter your age: "))
gender = input("Enter your gender (M/F/Other): ")
health_condition = input("Enter your health condition(s), comma separated (e.g., Diabetes,BP): ")
health_conditions = [h.strip() for h in health_condition.split(",")]

# Helper functions
def filter_foods(category, health_conditions):
    """Filter foods by category and health suitability"""
    query = "|".join(health_conditions)
    foods = df[
        (df['Category'] == category) &
        (df['Health Suitability'].str.contains(query, case=False, na=False))
    ]
    if len(foods) == 0:
        foods = df[df['Category'] == category]
    return foods

def generate_explanation(food_item, category, age, gender, health_conditions):
    """Generate AI explanation using OpenAI, considering age & gender"""
    prompt = f"""
    You are a professional Indian dietitian.
    Explain in 2-3 sentences why including {food_item} for {category} 
    is beneficial for someone of age {age}, gender {gender}, 
    with these health conditions: {', '.join(health_conditions)}.
    Keep it simple, friendly, and personalized.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def generate_meal(category, age, gender, health_conditions):
    """Generate one meal with AI explanation"""
    foods = filter_foods(category, health_conditions)
    meal_item = foods.sample(n=1).iloc[0]
    explanation = generate_explanation(meal_item['Food Item'], category, age, gender, health_conditions)
    
    return {
        'Food Item': meal_item['Food Item'],
        'Carbs': float(meal_item['Carbs']),
        'Protein': float(meal_item['Protein']),
        'Fat': float(meal_item['Fats']),
        'Calories': float(meal_item['Calories']),
        'Explanation': explanation
    }

# Load previous plans
if os.path.exists(previous_plans_path):
    with open(previous_plans_path, "r") as f:
        try:
            previous_plans = json.load(f)
        except json.JSONDecodeError:
            previous_plans = []
else:
    previous_plans = []

# Generate 1-day plan
daily_plan = {'Meals': {}, 'Total Calories': 0, 'Explanations': []}
total_calories = 0
day_explanations = []

for meal in meal_types:
    meal_info = generate_meal(meal, age, gender, health_conditions)
    daily_plan['Meals'][meal] = f"{meal_info['Food Item']} (Carbs: {meal_info['Carbs']}g, Protein: {meal_info['Protein']}g, Fat: {meal_info['Fat']}g, {meal_info['Calories']} kcal)"
    total_calories += meal_info['Calories']
    day_explanations.append(meal_info['Explanation'])

daily_plan['Total Calories'] = total_calories
daily_plan['Explanations'] = day_explanations

# Save plan to JSON
previous_plans.append(daily_plan)
with open(previous_plans_path, "w") as f:
    json.dump(previous_plans, f, indent=4)

# Display nicely
df_display = pd.DataFrame.from_dict(daily_plan['Meals'], orient='index', columns=['Meal Plan'])
df_display['Explanation'] = daily_plan['Explanations']
display(df_display)
print(f"\n✅ Total Calories: {total_calories} kcal")
print(f"\n✅ 1-day meal plan saved to JSON at {previous_plans_path}")

Enter your age:  23
Enter your gender (M/F/Other):  M
Enter your health condition(s), comma separated (e.g., Diabetes,BP):  Obesity, Diabetes


,Meal Plan,Explanation
Breakfast,"Oats (Carbs: 66.0g, Protein: 16.0g, Fat: 7.0g,...",Including oats for breakfast is a great choice...
Lunch,"Green Gram (Moong) (Carbs: 63.0g, Protein: 24....",Including green gram (moong) in your lunch is ...
Dinner,"Pumpkin (Carbs: 6.5g, Protein: 1.0g, Fat: 0.1g...",Including pumpkin in your dinner can be really...
Snack,"Flax Seeds (Carbs: 29.0g, Protein: 18.0g, Fat:...",Including flax seeds in your snacks is a fanta...



✅ Total Calories: 1296.0 kcal

✅ 1-day meal plan saved to JSON at C:\Users\polam\Downloads\AI Diet Planner\data\previous_1day_plans.json


In [17]:
import os
import json
import pandas as pd
import random
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display

# --- Load OpenAI API key ---
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- Paths ---
dataset_path = r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv"
weekly_plan_path = r"C:\Users\polam\Downloads\AI Diet Planner\data\previous_7day_plans.json"

# --- Load dataset ---
df = pd.read_csv(dataset_path)
meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# --- User input ---
age = int(input("Enter your age: "))
gender = input("Enter your gender (M/F/Other): ")
health_condition = input("Enter your health condition(s), comma separated (e.g., Diabetes,BP): ")
health_conditions = [h.strip() for h in health_condition.split(",")]

# --- Helper functions ---
def filter_foods(category, health_conditions, used_foods=[]):
    """Filter foods by category and health suitability, avoiding already used foods"""
    query = "|".join(health_conditions)
    foods = df[
        (df['Category'] == category) &
        (df['Health Suitability'].str.contains(query, case=False, na=False)) &
        (~df['Food Item'].isin(used_foods))
    ]
    if len(foods) == 0:
        # fallback to all foods in category if not enough
        foods = df[
            (df['Category'] == category) &
            (~df['Food Item'].isin(used_foods))
        ]
    if len(foods) == 0:
        # fallback to all foods in category including repeats if necessary
        foods = df[df['Category'] == category]
    return foods

def generate_explanation(food_item, category, age, gender, health_conditions):
    """Generate AI explanation for the meal"""
    prompt = f"""
    You are a professional Indian dietitian.
    Explain in 2-3 sentences why including {food_item} for {category} 
    is beneficial for someone of age {age}, gender {gender}, 
    with these health conditions: {', '.join(health_conditions)}.
    Keep it simple, friendly, and personalized.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def generate_meal(category, age, gender, health_conditions, used_foods):
    """Generate a meal with AI explanation"""
    foods = filter_foods(category, health_conditions, used_foods)
    meal_item = foods.sample(n=1).iloc[0]
    explanation = generate_explanation(meal_item['Food Item'], category, age, gender, health_conditions)
    
    return {
        'Food Item': meal_item['Food Item'],
        'Carbs': float(meal_item['Carbs']),
        'Protein': float(meal_item['Protein']),
        'Fat': float(meal_item['Fats']),
        'Calories': float(meal_item['Calories']),
        'Explanation': explanation
    }

# --- Load previous weekly plans ---
if os.path.exists(weekly_plan_path):
    with open(weekly_plan_path, "r") as f:
        try:
            previous_weekly_plans = json.load(f)
        except json.JSONDecodeError:
            previous_weekly_plans = []
else:
    previous_weekly_plans = []

# --- Generate 7-day plan ---
weekly_plan = []
used_foods = []

for day in range(1, 8):
    daily_plan = {'Day': f'Day {day}', 'Meals': {}, 'Total Calories': 0, 'Explanations': []}
    total_calories = 0
    day_explanations = []

    for meal in meal_types:
        meal_info = generate_meal(meal, age, gender, health_conditions, used_foods)
        daily_plan['Meals'][meal] = f"{meal_info['Food Item']} (Carbs: {meal_info['Carbs']}g, Protein: {meal_info['Protein']}g, Fat: {meal_info['Fat']}g, {meal_info['Calories']} kcal)"
        total_calories += meal_info['Calories']
        day_explanations.append(meal_info['Explanation'])
        used_foods.append(meal_info['Food Item'])  # mark as used to avoid repeats

    daily_plan['Total Calories'] = total_calories
    daily_plan['Explanations'] = day_explanations
    weekly_plan.append(daily_plan)

# --- Save weekly plan ---
previous_weekly_plans.append(weekly_plan)
with open(weekly_plan_path, "w") as f:
    json.dump(previous_weekly_plans, f, indent=4)

# --- Display nicely ---
for day_plan in weekly_plan:
    print(f"\n=== {day_plan['Day']} ===")
    df_display = pd.DataFrame.from_dict(day_plan['Meals'], orient='index', columns=['Meal Plan'])
    df_display['Explanation'] = day_plan['Explanations']
    display(df_display)
    print(f"Total Calories: {day_plan['Total Calories']} kcal")

Enter your age:  23
Enter your gender (M/F/Other):  F
Enter your health condition(s), comma separated (e.g., Diabetes,BP):  PCOD



=== Day 1 ===


,Meal Plan,Explanation
Breakfast,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g,...",Including eggs for breakfast can be a great ch...
Lunch,"Methi (Fenugreek) (Carbs: 7.0g, Protein: 4.0g,...",Including Methi (Fenugreek) in your lunch can ...
Dinner,"Mushroom (Carbs: 3.3g, Protein: 3.1g, Fat: 0.3...",Including mushrooms in your dinner can be real...
Snack,"Almonds (Carbs: 22.0g, Protein: 21.0g, Fat: 50...",Including almonds as a snack can be really ben...


Total Calories: 805.0 kcal

=== Day 2 ===


,Meal Plan,Explanation
Breakfast,"Yogurt (Carbs: 3.6g, Protein: 10.0g, Fat: 0.4g...",Including yogurt in your breakfast can be real...
Lunch,"Sweet Potato (Carbs: 26.0g, Protein: 2.0g, Fat...",Including sweet potato in your lunch is a grea...
Dinner,"Pumpkin (Carbs: 6.5g, Protein: 1.0g, Fat: 0.1g...",Including pumpkin in your dinner is a great ch...
Snack,"Beetroot (Carbs: 10.0g, Protein: 1.6g, Fat: 0....",Including beetroot in your snacks is a great c...


...

Total Calories: 771.0 kcal

=== Day 6 ===


,Meal Plan,Explanation
Breakfast,"Oats (Carbs: 66.0g, Protein: 16.0g, Fat: 7.0g,...",Including oats for breakfast is a fantastic ch...
Lunch,"Cottage Cheese (Low-fat) (Carbs: 3.4g, Protein...",Including low-fat cottage cheese in your lunch...
Dinner,"Broccoli (Carbs: 7.0g, Protein: 2.8g, Fat: 0.4...",Including broccoli in your dinner is a great c...
Snack,"Strawberries (Carbs: 7.7g, Protein: 0.7g, Fat:...",Including strawberries as a snack is a great c...


Total Calories: 553.0 kcal

=== Day 7 ===


,Meal Plan,Explanation
Breakfast,"Banana (Carbs: 23.0g, Protein: 1.0g, Fat: 0.3g...",Including a banana for breakfast is a great ch...
Lunch,"Paneer (Carbs: 1.2g, Protein: 18.0g, Fat: 20.0...",Including paneer in your lunch is a great choi...
Dinner,"Zucchini (Carbs: 3.1g, Protein: 1.2g, Fat: 0.3...",Including zucchini in your dinner can be reall...
Snack,"Apple (Carbs: 14.0g, Protein: 0.3g, Fat: 0.2g,...",Including apples as a snack is a great choice ...


Total Calories: 423.0 kcal


In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# --- Load API key ---
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- Chat history ---
chat_history = []

def generate_ai_response(user_message):
    """Generate AI response using OpenAI chat completion"""
    messages = [{"role": "system", "content": "You are a professional Indian dietitian chatbot. Give healthy diet suggestions based on user input."}]
    
    # Add previous conversation
    for chat in chat_history:
        messages.append({"role": "user", "content": chat['user']})
        messages.append({"role": "assistant", "content": chat['ai']})
    
    messages.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    
    return response.choices[0].message.content.strip()

# --- Chat loop ---
print("\n✅ Diet Chatbot Started! Type 'exit' to quit.\n")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye! Stay healthy 🍎")
        break
    
    ai_response = generate_ai_response(user_input)
    print(f"AI: {ai_response}\n")
    
    chat_history.append({"user": user_input, "ai": ai_response})



✅ Diet Chatbot Started! Type 'exit' to quit.



You:  Heyy


AI: Hello! How can I assist you with your dietary needs today?



You:  I am on my periods so can you share the diet plan for today!


AI: Of course! During your period, it's important to focus on foods that can help alleviate discomfort and keep your energy levels stable. Here’s a diet plan for today that may help:

### Breakfast:
- **Warm Oatmeal**: Cook oats with water or milk and top with sliced banana and a sprinkle of cinnamon. Oats are great for energy and can help reduce bloating.
- **Herbal Tea**: Ginger or chamomile tea can help soothe cramps.

### Mid-Morning Snack:
- **Fresh Fruits**: A small bowl of mixed fruits like oranges, apples, and berries for vitamins and hydration.

### Lunch:
- **Quinoa Salad**: Quinoa mixed with chopped cucumbers, tomatoes, bell peppers, and a dressing of olive oil, lemon juice, salt, and pepper. Quinoa is high in protein and fiber, which can help keep you full.
- **Lentil Soup**: A warm bowl of lentil soup for added iron and protein.

### Afternoon Snack:
- **Nuts and Seeds**: A small handful of almonds or walnuts. They provide healthy fats and can help with mood swings.

### D

You:  can you change the dinner


AI: Sure! Here’s an alternative dinner option that is nutritious and comforting:

### Dinner:
- **Palak Paneer**: A dish made with spinach (palak) and paneer (Indian cottage cheese). It’s rich in iron and calcium, which can help with fatigue.
- **Whole Wheat Roti or Brown Rice**: Serve it with a whole wheat roti or a small portion of brown rice for added fiber.
- **Cucumber Raita**: A side of cucumber raita (yogurt with grated cucumber and spices) can help cool the body and aid digestion.

This meal is wholesome and can provide the nutrients you need during your period. Let me know if you’d like more suggestions or adjustments!



You:  exit


Goodbye! Stay healthy 🍎


In [5]:
import pandas as pd
import random
from openai import OpenAI

# Load OpenAI API key
import os
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Load dataset
df = pd.read_csv(r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv")

# Define meal types
meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

def filter_foods(category):
    """Filter foods by category and available locally"""
    foods = df[df['Category'] == category]
    return foods

def generate_meal_response(category):
    """Pick a meal and generate explanation using OpenAI"""
    foods = filter_foods(category)
    meal_item = foods.sample(n=1).iloc[0]

    # Prompt for OpenAI to generate explanation
    prompt = f"""
    You are a professional Indian dietitian.
    Explain in 2-3 sentences why {meal_item['Food Item']} is a good choice for {category}.
    Keep it simple and friendly.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    explanation = response.choices[0].message.content.strip()

    return f"{meal_item['Food Item']} ({category}) - {explanation}"

# Chat loop
print("🟢 Diet Chatbot is ready! Type 'exit' to quit.")
while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ['exit', 'quit']:
        print("👋 Goodbye!")
        break
    
    # Check if user mentions a meal type
    found = False
    for meal in meal_types:
        if meal.lower() in user_input.lower():
            print("\nAI:", generate_meal_response(meal))
            found = True
            break
    
    if not found:
        # General response from OpenAI
        prompt = f"""
        You are a professional Indian dietitian chatbot.
        Respond to this user input: "{user_input}"
        Suggest meals and give explanations based on Indian food items.
        """
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        print("\nAI:", response.choices[0].message.content.strip())

🟢 Diet Chatbot is ready! Type 'exit' to quit.



You:  i am on periods diet plan for today



AI: Certainly! A diet during your periods should focus on foods that provide essential nutrients, help reduce bloating, and alleviate cramps. Here’s a suggested meal plan for today, incorporating traditional Indian foods:

### Breakfast:
**1. Moong Dal Chilla (Green Gram Pancake)**  
- **Why:** High in protein and easy to digest, moong dal is great for keeping your energy levels stable. You can pair it with a side of mint chutney for added flavor.

**2. A Glass of Warm Ginger Tea**  
- **Why:** Ginger helps reduce inflammation and ease menstrual cramps. Opt for a tea made with fresh ginger, and add a bit of honey for sweetness.

### Mid-Morning Snack:
**1. Fresh Fruits (like bananas or apples)**  
- **Why:** Fruits are rich in vitamins and minerals. Bananas, in particular, can help in reducing bloating and provide potassium, which may help alleviate cramps.

### Lunch:
**1. Palak Paneer (Spinach with Cottage Cheese)**  
- **Why:** Spinach is rich in iron, which is essential during men


You:  exit


👋 Goodbye!
